In [1]:
import pandas as pd

In [7]:
tweets = pd.read_csv('../data/TweetsCOV19.csv')

/tmp/ipykernel_175958/4023932799.py:1: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  tweets = pd.read_csv('../data/TweetsCOV19.csv')


In [22]:
print(tweets.loc[5]["TweetText"])
print(tweets.loc[5]["Entities"])

Looking like it may be a fall-like weekend coming up! Dry cold front looks to move through the area Friday with hig… https://t.co/qOyZm9sS2B
cold front:Cold_front:-1.3939860168378515;


In [25]:
tweets.loc[675372]["TweetText"]

"@DorseyFilm @KyleClark So it's selfish of me to not want a vaccine that has been rushed and who the hell knows what's in it?  And I'm not self-hating.  ;-) Thanks for your psychoanalysis though."

In [26]:
display(tweets)

,Unnamed: 0,TweetId,Username,Timestamp,NoFollowers,NoFriends,NoRetweets,NoFavorites,Entities,Sentiment,Mentions,Hashtags,URLs,TweetText,UserLocation
0,0,1178793230223183872,bf05d1888dd974fa4a8679c25e2ead03,2019-09-30 22:06:21+00:00,5018.0,1933.0,0.0,0.0,vaccine:Vaccine:-2.6651530673745762;anti vaxxe...,2 -1,null;,null;,https://goo.gl/fb/uoeiPk:-:,From my blog: Californians support vaccine law...,"Los Angeles, CA"
1,1,1178798309491822592,00695cf79d60e86a5d0f872f4358337a,2019-09-30 22:26:32+00:00,9009.0,4943.0,26.0,89.0,people of northern ireland:People_of_Northern_...,2 -1,null;,VFS,null;,Secretary of State should recall Stormont next...,United Kingdom
2,2,1178806730358059008,6c9c28a4e449e0fe0d4d2ff449b8d6bc,2019-09-30 23:00:00+00:00,237005.0,559.0,2.0,3.0,ied:Improvised_explosive_device:-1.76224648952...,2 -3,null;,null;,https://www.fox5vegas.com/toby-keith-and-a-vet...,"While serving in Afghanistan in 2010, Marine C...",Las Vegas
3,3,1178807288171130880,000000e6bded7cdeaffee6928df45050,2019-09-30 23:02:13+00:00,1658.0,23.0,2.0,4.0,null;,1 -1,null;,null;,null;,witch vixen season starts tomorrow. you all sh...,irl
4,4,1178809851843006464,0fc683f42b4929419e796c5d0d97ce1b,2019-09-30 23:12:24+00:00,14231403.0,43.0,3.0,10.0,cgtn:China_Global_Television_Network:-1.951800...,1 -1,null;,PRC70 NationalDay2019,https://news.cgtn.com/news/2019-09-26/Live-Gra...,"CGTN on the scene: \n\nAround 15,000 troops, 3...","Beijing, China"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675370,11803,1266652472119971840.0,2c247493db0e5f40ae8f5a048414b7e8,2020-05-30 08:47:37+00:00,9396.0,9438.0,1.0,1.0,smh:The_Sydney_Morning_Herald:-2.6918118260331...,2 -3,null;,Chinazi HongKong China. Trump SMH,null;,WTF is this supposed to mean? These #Chinazi s...,America! The land of the free!
675371,11804,1262836622363590656.0,d2323ad6e94afab3963dd0d8f43316b8,2020-05-19 20:04:47+00:00,11476410.0,8.0,313.0,1685.0,hydroxychloroquine:Hydroxychloroquine:-1.08359...,2 -1,null;,null;,https://www.theonion.com/growing-drug-toleranc...,Growing Drug Tolerance Drives Trump To Buy Bla...,NaN
675372,11805,1261417719892566016.0,2066652744987178573fce8b5953e325,2020-05-15 22:06:35+00:00,4590.0,4920.0,1.0,3.0,psychoanalysis:Psychoanalysis:-1.7095004037976...,2 -4,DorseyFilm KyleClark,null;,null;,@DorseyFilm @KyleClark So it's selfish of me t...,"Colorado, USA"
675373,11806,1256806277990297600.0,714dcfa47a15f6a1cbbf3eefcaf1a13f,2020-05-03 04:42:21+00:00,7404.0,3442.0,0.0,0.0,unicef:UNICEF:-1.1160110760489719;psychologica...,1 -4,UNICEF,FGM,https://www.unicef.org/sudan/press-releases/su...,"""It’s estimated that 87% of Sudanese females h...",@raphaellerl@piaille.fr 🐘
